# Assignment 2: Entity Resolution (Part 1)

## Objective

Data scientists often spend 80% of their time on [data preparation](https://www.infoworld.com/article/3228245/the-80-20-data-science-dilemma.html). If your career goal is to become a data scientist, you have to master data cleaning and data integration skills. In this assignment, you will learn how to solve the Entity Resolution (ER) problem, a very common problem in data cleaning and integration. After completing this assignment, you should be able to answer the following questions:

1. What is ER?
2. What are the applications of ER in data integration and cleaning? 
3. How to avoid $n^2$ comparisons? 
4. How to compute Jaccard Similarity?
5. How to evaluate an ER result?

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please follow python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

The data for Assignment 2 (Part 1 and Part 2) can be downloaded from [A2-data.zip](A2-data.zip).

## Overview

ER is defined as finding different records that refer to the same real-world entity, e.g., iPhone 4-th generation vs. iPhone four. It is central to data integration and cleaning. In this assignment, you will learn how to apply ER in a data integration setting. But the program that you are going to write can be easily extended to a data-cleaning setting, being used to detect _duplication values_.   

Imagine that you want to help your company's customers to buy products at a cheaper price. In order to do so, you first write a [web scraper](https://nbviewer.jupyter.org/github/sfu-db/bigdata-cmpt733/blob/master/Assignments/A1/A1-1.ipynb) to crawl product data from Amazon.com and Google Shopping, respectively, and then integrate the data together. Since the same product may have different representations in the two websites, you are facing an ER problem. 

Existing ER techniques can be broadly divided into two categories: similarity-based (Part 1) and learning-based (Part 2). 


## Similarity Join

Unlike a learning-based technique, a similarity-based technique (a.k.a similarity join) does not need any label data. It first chooses a similarity function and a threshold, and then returns the record pairs whose similarity values are above the threshold. These returned record pairs are thought of as matching pairs, i.e., referring to the same real-world entity. 

Depending on particular applications, you may need to choose different similarity functions. In this assignment, we will use Jaccard similarity, i.e., $\textsf{Jaccard}(r, s) = \big|\frac{r \cap s}{r \cup s}\big|$. Here is the formal definition of this problem.

> **Jaccard-Similarity Join**: Given two DataFrames, R and S, and a threshold $\theta \in (0, 1]$, the jaccard-similarity join problem aims to find all record pairs $(r, s) \in R \times S$ such that $\textsf{Jaccard}(r, s) \geq \theta$  

To implement similarity join, you need to address the following challenges:

1. Jaccard is used to quantify the similarity between two sets instead of two records. You need to convert each record to a set.

2. A naive implementation of similarity join is to compute Jaccard for all $|R \times S|$ possible pairs. Imagine R and S have one million records. This requires doing 10^12 pair comparisons, which is extremely expensive. Thus, you need to know how to avoid n^2 comparisons. 

3. The output of ER is a set of matching pairs, where each pair is considered as referring to the same real-world entity. You need to know how to evaluate the quality of an ER result.

Next, you will be guided to complete four tasks. After finishing these tasks, I suggest you going over the above challenges again, and understand how they are addressed.

Read the code first, and then implement the remaining four functions: `preprocess_df`, `filtering`, `verification`, and `evaluate` by doing Tasks A-D, respectively.

``` python
# similarity_join.py
import re
import pandas as pd

class SimilarityJoin:
    def __init__(self, data_file1, data_file2):
        self.df1 = pd.read_csv(data_file1)
        self.df2 = pd.read_csv(data_file2)
          
    def preprocess_df(self, df, cols): 
        """
            Write your code!
        """ 
    
    def filtering(self, df1, df2):
        """
            Write your code!
        """
      
    def verification(self, cand_df, threshold):
        """
            Write your code!
        """
        
    def evaluate(self, result, ground_truth):
        """
            Write your code!
        """
        
    def jaccard_join(self, cols1, cols2, threshold):
        new_df1 = self.preprocess_df(self.df1, cols1)
        new_df2 = self.preprocess_df(self.df2, cols2)
        print ("Before filtering: %d pairs in total" %(self.df1.shape[0] *self.df2.shape[0])) 
        
        cand_df = self.filtering(new_df1, new_df2)
        print ("After Filtering: %d pairs left" %(cand_df.shape[0]))
        
        result_df = self.verification(cand_df, threshold)
        print ("After Verification: %d similar pairs" %(result_df.shape[0]))
        
        return result_df
       
        

if __name__ == "__main__":
    er = SimilarityJoin("Amazon_sample.csv", "Google_sample.csv")
    amazon_cols = ["title", "manufacturer"]
    google_cols = ["name", "manufacturer"]
    result_df = er.jaccard_join(amazon_cols, google_cols, 0.5)

    result = result_df[['id1', 'id2']].values.tolist()
    ground_truth = pd.read_csv("Amazon_Google_perfectMapping_sample.csv").values.tolist()
    print ("(precision, recall, fmeasure) = ", er.evaluate(result, ground_truth))
```


The program will output the following when running on the sample data:


> Before filtering: 256 pairs in total

> After Filtering: 84 pairs left

> After Verification: 6 similar pairs

> (precision, recall, fmeasure) =  (1.0, 0.375, 0.5454545454545454)


### Task A. Data Preprocessing (Record --> Token Set)

Since Jaccard needs to take two sets as input, your first job is to preprocess DataFrames by transforming each record into a set of tokens. Please implement the following function.   

```python
def preprocess_df(self, df, cols): 
    """ 
        Input: $df represents a DataFrame
               $cols represents the list of columns (in $df) that will be concatenated and be tokenized

        Output: Return a new DataFrame that adds the "joinKey" column to the input $df

        Comments: The "joinKey" column is a list of tokens, which is generated as follows:
                 (1) concatenate the $cols in $df; 
                 (2) apply the tokenizer to the concatenated string
        Here is how the tokenizer should work:
                 (1) Use "re.split(r'\W+', string)" to split a string into a set of tokens
                 (2) Convert each token to its lower-case
    """ 
```

For the purpose of testing, you can compare your outputs with new_df1 and new_df2 that can be found from the `Amazon-Google-Sample` folder.

In [8]:
import pandas as pd
pd.set_option('display.max_colwidth', 255)

In [9]:
import pandas as pd
import re

amazon_cols = ["title", "manufacturer"]
google_cols = ["name", "manufacturer"]

# amazon = pd.read_csv('A2-data/part1-similarity-join/Amazon-Google-Sample/Amazon_sample.csv')
# google = pd.read_csv('A2-data/part1-similarity-join/Amazon-Google-Sample/Google_sample.csv')

amazon = pd.read_csv('A2-data/part1-similarity-join/Amazon-Google/Amazon.csv')
google = pd.read_csv('A2-data/part1-similarity-join/Amazon-Google/Google.csv')

google['manufacturer'] = google['manufacturer'].fillna('')

# display(amazon)
# display(google)

amazon['joinKey'] = amazon[amazon_cols].apply(' '.join, axis=1)
google['joinKey'] = google[google_cols].apply(' '.join, axis=1)

amazon['joinKey'] = amazon['joinKey'].apply(lambda x: list(
	filter(
		None, 
		list(
			set(
				[x.lower() for x in re.split(r'\W+', x.rstrip('.'))]
			)
		)
	)
))
google['joinKey'] = google['joinKey'].apply(lambda x: list(filter(None, list(set([x.lower() for x in re.split(r'\W+', x.rstrip('.'))])))))

amazon.to_csv('df1.csv', index=False)
google.to_csv('df2.csv')

print(amazon.shape)
print(google.shape)

display(amazon.head(5))
display(google.head(5))


(1363, 7)
(3226, 7)


,Unnamed: 0,id,title,description,manufacturer,price,joinKey
0,0,b000jz4hqo,clickart 950 000 - premier image pack (dvd-rom),NaN,broderbund,0.00,"[broderbund, image, 950, 000, dvd, rom, clickart, pack, premier]"
1,1,b0006zf55o,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops and desktops,computer associates,0.00,"[lap, international, associates, desktop, arcserve, computer, 30pk, oem, ca]"
2,2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),NaN,victory multimedia,0.00,"[case, victory, noah, 3, ark, 8, s, ages, multimedia, center, activity, jewel]"
3,3,b000g80lqo,peachtree by sage premium accounting for nonprofits 2007,peachtree premium accounting for nonprofits 2007 is the affordable easy to use accounting solution that provides you with donor/grantor management. if you're like most nonprofit organizations you're constantly striving to maximize each and every dolla...,sage software,599.99,"[software, peachtree, 2007, sage, by, accounting, for, nonprofits, premium]"
4,4,b0006se5bq,singing coach unlimited,singing coach unlimited - electronic learning products (win me nt 2000 xp),carry-a-tune technologies,99.99,"[technologies, a, tune, unlimited, carry, coach, singing]"


,Unnamed: 0,id,name,description,manufacturer,price,joinKey
0,0,http://www.google.com/base/feeds/snippets/11125907881740407428,learning quickbooks 2007,learning quickbooks 2007,intuit,38.99,"[learning, 2007, intuit, quickbooks]"
1,1,http://www.google.com/base/feeds/snippets/11538923464407758599,superstart! fun with reading & writing!,fun with reading & writing! is designed to help kids learn to read and write better through exercises puzzle-solving creative writing decoding and more!,,8.49,"[writing, fun, superstart, reading, with]"
2,2,http://www.google.com/base/feeds/snippets/11343515411965421256,qb pos 6.0 basic software,qb pos 6.0 basic retail mngmt software. for retailers who need basic inventory sales and customer tracking.,intuit,637.99,"[software, intuit, pos, 6, 0, qb, basic]"
3,3,http://www.google.com/base/feeds/snippets/12049235575237146821,math missions: the amazing arcade adventure (grades 3-5),save spectacle city by disrupting randall underling's plan to drive all the stores out of business and take over the city. solve real-world math challenges in the uniquely entertaining stores and make theme successful again.,,12.95,"[amazing, missions, adventure, 3, arcade, the, grades, math, 5]"
4,4,http://www.google.com/base/feeds/snippets/12244614697089679523,production prem cs3 mac upgrad,adobe cs3 production premium mac upgrade from production studio premium or standard,adobe software,805.99,"[software, production, upgrad, mac, cs3, prem, adobe]"


### Task B. Filtering Obviously Non-matching Pairs

To avoid $n^2$ pair comparisons, ER algorithms often follow a filtering-and-verification framework. The basic idea is to first filter obviously non-matching pairs and then only verify the remaining pairs.  

In Task B, your job is to implement the <font color="blue">filtering</font> function. This function will filter all the record pairs whose joinKeys do not share any token. This is because based on the definition of Jaccard, we can deduce that **if two sets do not share any element (i.e., $r\cap s = \phi$), their Jaccard similarity values must be zero**. Thus, we can safely remove them. 

```python
def filtering(self, df1, df2):
    """ 
        Input: $df1 and $df2 are two input DataFrames, where each of them 
               has a 'joinKey' column added by the preprocess_df function

        Output: Return a new DataFrame $cand_df with four columns: 'id1', 'joinKey1', 'id2', 'joinKey2',
                where 'id1' and 'joinKey1' are from $df1, and 'id2' and 'joinKey2'are from $df2.
                Intuitively, $cand_df is the joined result between $df1 and $df2 on the condition that 
                their joinKeys share at least one token. 

        Comments: Since the goal of the "filtering" function is to avoid n^2 pair comparisons, 
                  you are NOT allowed to compute a cartesian join between $df1 and $df2 in the function. 
                  Please come up with a more efficient algorithm (see hints in Lecture 2). 
    """
```

For the purpose of testing, you can compare your output with cand_df that can be found from the `Amazon-Google-Sample` folder.

In [14]:
amazon_short = amazon[['id', 'joinKey']]
google_short = google[['id', 'joinKey']]
amazon_exploded = amazon_short.set_index('id') \
								.apply(pd.Series.explode) \
								.reset_index() \
								.set_index('joinKey')
google_exploded = google_short.set_index('id') \
								.apply(pd.Series.explode) \
								.reset_index() \
								.set_index('joinKey')

# print(amazon_exploded.shape)
# print(google_exploded.shape)

common_tokens = amazon_exploded.join(google_exploded, how='inner', lsuffix='1', rsuffix='2')

# common_tokens.reset_index(drop=True).drop_duplicates()

common_tokens_with_count = common_tokens.groupby(['id1', 'id2']) \
										.size() \
										.reset_index(name='r_intersect_s')

cand_df = common_tokens_with_count.merge(amazon_short.rename(columns={'id':'id1', 'joinKey':'joinKey1'}), on='id1') \
									.merge(google_short.rename(columns={'id':'id2', 'joinKey':'joinKey2'}), on='id2') \
									[['id1', 'joinKey1', 'id2', 'joinKey2']]


display(cand_df.head(5))
print(cand_df.shape)

cand_df.to_csv("cand_df.csv", index=False)


,id1,joinKey1,id2,joinKey2
0,1931102953,"[immersion, language, topics, win, sign, entertainment, american, mac, instant]",http://www.google.com/base/feeds/snippets/10004325115759097777,"[italian, dlux, instant, 2, immers]"
1,b00004t2un,"[skills, victory, 10, ages, multimedia, 0, sales, 2]",http://www.google.com/base/feeds/snippets/10004325115759097777,"[italian, dlux, instant, 2, immers]"
2,b000053f92,"[activision, sega, smash, pack, 2]",http://www.google.com/base/feeds/snippets/10004325115759097777,"[italian, dlux, instant, 2, immers]"
3,b00005birs,"[math, school, 2, zone]",http://www.google.com/base/feeds/snippets/10004325115759097777,"[italian, dlux, instant, 2, immers]"
4,b00005ld1r,"[web, international, 3, 2, publisher, computer, associates, model, 5]",http://www.google.com/base/feeds/snippets/10004325115759097777,"[italian, dlux, instant, 2, immers]"


(743382, 4)


### Task C. Computing Jaccard Similarity for Survived Pairs

In the second phase of the filtering-and-verification framework, we will compute the Jaccard similarity for each survived pair and return those pairs whose jaccard similarity values are no smaller than the specified threshold.

In Task C, your job is to implement the <font color="blue">verification</font> function. This task looks simple, but there are a few small "traps". 


```python
def verification(self, cand_df, threshold):
        """ 
            Input: $cand_df is the output DataFrame from the 'filtering' function. 
                   $threshold is a float value between (0, 1] 

            Output: Return a new DataFrame $result_df that represents the ER result. 
                    It has five columns: id1, joinKey1, id2, joinKey2, jaccard 

            Comments: There are two differences between $cand_df and $result_df
                      (1) $result_df adds a new column, called jaccard, which stores the jaccard similarity 
                          between $joinKey1 and $joinKey2
                      (2) $result_df removes the rows whose jaccard similarity is smaller than $threshold 
        """
```

For the purpose of testing, you can compare your output with result_df that can be found from the `Amazon-Google-Sample` folder.

In [11]:
cand_df['jaccard'] = cand_df.apply(lambda x: (len(list(set(x[1]).intersection(set(x[3])))) / len(list(set(x[1]).union(set(x[3]))))), axis=1)
# display(cand_df.head(10))

result_df = cand_df[cand_df['jaccard'] >= 0.5]

display(result_df.head(10))

result_df.to_csv('result_df.csv', index=False)

,id1,joinKey1,id2,joinKey2,r_intersect_s,jaccard
450,b0002dlrru,"[immersion, topics, italian, entertainment, platinum, instant]",http://www.google.com/base/feeds/snippets/10066664236286478958,"[immersion, topics, italian, 0, entertainment, instant, 2]",5,0.625000
475,b0007yepxc,"[immersion, topics, italian, 0, large, entertainment, v2, instant, box]",http://www.google.com/base/feeds/snippets/10066664236286478958,"[immersion, topics, italian, 0, entertainment, instant, 2]",6,0.600000
480,b0008ews6w,"[immersion, topics, japanese, 0, entertainment, instant, 2]",http://www.google.com/base/feeds/snippets/10066664236286478958,"[immersion, topics, italian, 0, entertainment, instant, 2]",6,0.750000
481,b0008ews9o,"[immersion, topics, 0, entertainment, french, instant, 2]",http://www.google.com/base/feeds/snippets/10066664236286478958,"[immersion, topics, italian, 0, entertainment, instant, 2]",6,0.750000
482,b0008ewsai,"[immersion, topics, italian, 0, entertainment, instant, 2]",http://www.google.com/base/feeds/snippets/10066664236286478958,"[immersion, topics, italian, 0, entertainment, instant, 2]",7,1.000000
483,b0008ewsf8,"[immersion, topics, spanish, 0, entertainment, instant, 2]",http://www.google.com/base/feeds/snippets/10066664236286478958,"[immersion, topics, italian, 0, entertainment, instant, 2]",6,0.750000
500,b0009x6qew,"[immersion, topics, italian, 0, large, entertainment, instant, v2, deluxe, box]",http://www.google.com/base/feeds/snippets/10066664236286478958,"[immersion, topics, italian, 0, entertainment, instant, 2]",6,0.545455
526,b000bgpqos,"[immersion, topics, japanese, 2, 0, entertainment, instant, deluxe]",http://www.google.com/base/feeds/snippets/10066664236286478958,"[immersion, topics, italian, 0, entertainment, instant, 2]",6,0.666667
527,b000bgpqr0,"[immersion, topics, german, 2, 0, entertainment, instant, deluxe]",http://www.google.com/base/feeds/snippets/10066664236286478958,"[immersion, topics, italian, 0, entertainment, instant, 2]",6,0.666667
528,b000bgtv80,"[immersion, topics, spanish, 2, 0, entertainment, instant, deluxe]",http://www.google.com/base/feeds/snippets/10066664236286478958,"[immersion, topics, italian, 0, entertainment, instant, 2]",6,0.666667


### Task D. Evaluating an ER result

How should we evaluate an ER result? Before answering this question, let's first recall what the ER result looks like. The goal of ER is to identify all matching record pairs. Thus, the ER result should be a set of identified matching pairs, denoted by R. One thing that we want to know is that what percentage of the pairs in $R$ that are truly matching? This is what Precision can tell us. Let $T$ denote the truly matching pairs in $R$. Precision is defined as:
$$Precision = \frac{|T|}{|R|}$$

In addition to Precision, another thing that we care about is that what percentage of truly matching pairs that are identified. This is what Recall can tell us. Let $A$ denote the truly matching pairs in the entire dataset. Recall is defined as: 

$$Recall = \frac{|T|}{|A|}$$

There is an interesting trade-off between Precision and Recall. As more and more pairs that are identified as matching, Recall increases while Precision potentially decreases. For the extreme case, if we return all the pairs as matching pairs, we will get a perfect Recall (i.e., Recall = 100%) but precision will be the worst. Thus, to balance Precision and Recall, people often use FMeasure to evaluate an ER result:

$$FMeasure = \frac{2*Precision*Recall}{Precision+Recall}$$

In Task D, you will be given an ER result as well as the ground truth that tells you what pairs are truly matching. Your job is to calculate Precision, Recall and FMeasure for the result. 
```python
def evaluate(self, result, ground_truth):
    """ 
        Input: $result is a list of matching pairs identified by the ER algorithm
               $ground_truth is a list of matching pairs labeld by humans

        Output: Compute precision, recall, and fmeasure of $result based on $ground_truth, and
                return the evaluation result as a triple: (precision, recall, fmeasure)

    """
```


In [13]:
# ground_truth = pd.read_csv('A2-data/part1-similarity-join/Amazon-Google-Sample/Amazon_Google_perfectMapping_sample.csv')
ground_truth = pd.read_csv('A2-data/part1-similarity-join/Amazon-Google/Amazon_Google_perfectMapping.csv')

identified_matching_pairs = result_df.shape[0]
truly_matching_pairs_in_dataset = ground_truth.shape[0]

truly_matching_pairs = 0

for index, rows in result_df.iterrows():
	amazon_id, google_id = rows[['id1', 'id2']]
	if (ground_truth['idAmazon'] == amazon_id).any():
		google_id_orig = ground_truth.loc[ground_truth['idAmazon'] == amazon_id,'idGoogle'].iloc[0]
		if google_id == google_id_orig:
			truly_matching_pairs += 1
		
precision = truly_matching_pairs/identified_matching_pairs

recall = truly_matching_pairs/truly_matching_pairs_in_dataset

fmeasure = (2 * precision * recall) / (precision + recall)

print ("Before filtering: %d pairs in total" %(amazon.shape[0] * google.shape[0])) 
print ("After Filtering: %d pairs left" %(cand_df.shape[0]))
print ("After Verification: %d similar pairs" %(result_df.shape[0]))

(precision, recall, fmeasure)

Before filtering: 4397038 pairs in total
After Filtering: 743382 pairs left
After Verification: 1823 similar pairs


(0.3406472846955568, 0.4776923076923077, 0.39769452449567727)

## Submission

Implement `preprocess_df`, `filtering`, `verification`, and `evaluate` functions in `similarity_join.py`. Submit your code file (`similarity_join.py`)  to the CourSys activity Assignment 2.